In this notebook we do clustering on the small version of gene network that contains only Sfari genes, and save the clusters for further analysis. Can be used for other tissues in the same way.

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import community

    The input file has to describe an undirected, weighted graph. 
    It contains info about the edges, has to be in the following format: 
        3 columns, first two are node IDs, the third one is the weight.
        Node IDs should be Entrez IDs of genes, weights are floats 
        (functional interaction between genes).
        
    Example rows:
        
    Gene1 Gene2 Weight
    9976  9987  0.134438
    998   9986  0.158842
    
    Network used in this Notebook: GIANT Network (Troanskaya Labs - Princeton / Flatiron):
    https://hb.flatironinstitute.org/download
    Top Edges version

In [ ]:
# Read GIANT Network for brain tissue - Top Edges
df_giant = pd.read_csv("Data/brain_top", sep='\t', names = ["g1", "g2", "w"])

In [ ]:
# List of Sfari genes
df_sfari = pd.read_csv("Data/sfari-converted.csv", sep='\t', 
                       usecols=["symbol", "score", "entrez"])

# remove genes with null score
df_sfari = df_sfari[df_sfari.score.notnull()]
print(str(df_sfari.shape) + "\n")

# remove genes with 6 score
df_sfari = df_sfari[df_sfari.score != 6.0]
print(str(df_sfari.shape) + "\n")

# remove genes with 5 score
df_sfari = df_sfari[df_sfari.score != 5.0]
print(str(df_sfari.shape) + "\n")

In [ ]:
gene_list = df_sfari.entrez.to_list()
gene_list = [int(a) for a in gene_list if str(a) != 'nan']
print(len(gene_list))

In [ ]:
# from GIANT network keep only those edges where both genes belong to the sfari list.
df_g = df_giant[df_giant['g1'].isin(gene_list) & df_giant['g2'].isin(gene_list)] 

In [ ]:
# build networkx graph
G = nx.Graph()
for row in df_g.itertuples():
    G.add_edge(row[1], row[2], weight=row[3])

In [ ]:
# quick look at the graph
print(nx.is_connected(G))
print(len(G.edges()))
print(len(G.nodes()))

print(df_g["w"].mean())

# degree based on weight sum
gdegree = G.degree(weight="weight")
print(np.mean(list(dict(gdegree).values())))

In [6]:
# node strength (based on edge weights) distribution histogram
plt.hist(list(dict(gdegree).values()), bins=50)

In [ ]:
# Louvain community detection - https://python-louvain.readthedocs.io/en/latest/
partition = community.best_partition(G)

for com in set(partition.values()) :
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    
    outstr = "small_giant_brain_louvain/giant-" + str(com) + ".txt"
    
    # uncomment to save gene list to file
    # np.savetxt(outstr, list_nodes, "%i")
    
    # see the size of each partition
    print(str(com) + ": " + str(len(list_nodes)))



In [ ]:
# Need to save node strength for each gene, to visualize in boxplot
# (same info as 'gdegree', but different format for convenience)
wdegree = [b for (a,b) in G.degree(weight="weight")]
print(len(wdegree))

# uncomment to save file
# np.savetxt("boxplot_degree/brain-degrees.txt", wdegree, fmt='%1.3f')